In this notebook we will write a generalized version of Greedy Best First Search methods that can work with any problem once it is formulated. Then, the key of the solution will be generating proper problem formulations and the proper heuristic function.


## Frontier operation Functions
In GBFS methods the extraction is not based on their arrival to the queue order, but on the similarity to the heuristic function.



## PROBLEM FORMULATION
The problem was formulated as a Python class, with the actions, restrictions and functions integrated and always the same name, so they can be re-coded for different problems and the search methods work the same. We start creating the class for River crossing problem. Creating other problems will only require changing some parts of this class. We will start with P8 problem: It is exactly the same but we need to code the


In [ ]:
import numpy as np
class ProblemP8 ():
    #attributes of the class are empty
    name = ""
    initial_state = {}
    goal_state = {}
    actions = []
    rows = 3
    columns= 3

    def __init__(self): ## init method is the constructor.

        # normally we would go with a random initialization
        numbers= np.arange(self.rows*self.columns)
        np.random.shuffle(numbers)

        #numbers =np.array([1,2,5,3,4,8,0,6,7])##--->> this configuration only requires depth 6 solution
        #numbers =np.array([1,2,5,3,6,7,0,4,8]) #---<>> depth 11 solution
        numbers =np.array([8,6,7,4,5,3,2,1,0])
        self.name ="8PuzzleProblem"
        self.initial_state = numbers.reshape(self.rows, self.columns)
        self.goal_state = np.arange(self.rows*self.columns).reshape(self.rows, self.columns)
        self.actions = ["Up", "Down", "Left", "Right"]


    #we need this in case we want other size
    def set_row_cols (self, rows, cols):
        self.rows=rows
        self.columns=cols

    def is_final_state(self, state):
        return np.array_equal(state, self.goal_state)

    #####
    def is_applicable (self, state, action):
        #find where is 0 in the state to know if the change is applicable:
        row = np.where(state==0)[0][0] +1
        col = np.where(state==0)[1][0] +1
        if action == "Up":
          return row!=1
        if action == "Down":
          return row!=self.rows
        if action == "Left":
          return col!=1
        if action == "Right":
          return col!=self.columns


    # this function should exist for all problems, but its internal operation should be changed per problem
    # in this case, the applicability is checked in the expansion function, so the effect of a change is always produced
    def effect (self, state, action):
        row = np.where(state==0)[0][0]
        col = np.where(state==0)[1][0]
        result = state.copy()
        if action == "Up":
            result[row-1,col]=state[row,col]
            result[row,col]=state[row-1,col]

        if action == "Down":
          result[row+1,col]=state[row,col]
          result[row,col]=state[row+1,col]

        if action == "Left":
          result[row,col-1]=state[row,col]
          result[row,col]=state[row,col-1]

        if action == "Right":
          result[row,col+1]=state[row,col]
          result[row,col]=state[row,col+1]

        return result


  ## NOW COST AND EVALUATION ARE RELEVANT. EVALUATION IS THE HEURISTIC FUNCTION THAT WILL SAY HOW SIMILAR IS A SOLUTION TO FINAL ONE.
  ### in this case, all actions have the same cost, moving 1
    def get_cost(self, action, state):
        return 1
  ### but the evaluation should be some function that tells us how different is the solution

    def get_evaluation (self, state):
      # returns the sum of elements that are different to the goal state: we want to bring this evaluation to 0.
      return sum(sum(state!=self.goal_state))


In [ ]:
p8problem = ProblemP8()
print(p8problem.initial_state)

print (p8problem.get_evaluation(p8problem.initial_state))
print (p8problem.get_evaluation(p8problem.goal_state))

[[8 6 7]
 [4 5 3]
 [2 1 0]]
9
0


# Expansion Function
Expansion function is the same as before


It receives the problem, with the functions and actions, and a NODE, which includes the following:

* state	(current state)
* parent node (parent node, starts with empty)
* actions (list of actions that led here)
* cost (starting in 0)
* depth (starting in 0)
* evaluation <----- h(n)





In [ ]:
# instead of updating the frontier, the expand function only creates a list of child of possible nodes given a current node
# it does not receive states, but NODES, dictionaries that include other information besides state
def expand (node, problem):
    new_nodes = []
    possible_actions = problem.actions
    for action in possible_actions:
        if problem.is_applicable(node["state"], action):
            new_state= problem.effect (node["state"], action)
            new_node = {}
            new_node["state"]=new_state
            new_node["parent_node"]=node
            new_node["actions"]=node["actions"] + [action]
            new_node["cost"]=problem.get_cost(action, node["state"])
            new_node["depth"]=node["depth"]+1
            new_node["evaluation"]=problem.get_evaluation(new_state)
            new_nodes.append (new_node)
    return new_nodes

# UNINFORMED SEARCH METHODS


## BFS - Breadth First Search

```
1. Make a node with the initial problem state
2. Insert node into the frontier data structure
3. WHILE final state not found AND frontier is not empty DO
  3.1 Remove first node from the frontier
  3.2 IF node contains final state THEN final state found
  3.3 IF node doesn’t contain final state THEN
     3.3.1 EXPAND node’s state
     3.3.2 Insert successor nodes into frontier
4. IF final state found THEN
  4.1  RETURN sequence of actions found
5. ELSE  “solution not found”
```



In [ ]:
def BFS(problem):
    # result dictionary
    result = {"method":"BFS", "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    # problem = Problem()
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
    frontier = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
    while len (frontier)>0: #if we have elements in the frontier...
         # 3.1. get first element of frontier and delete it
        node = frontier[0]


        frontier = frontier[1:]
        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            frontier.append(n)

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)





## BFS-g

In [ ]:
def BFS_g(problem):
    # result dictionary
    result = {"method":"BFS_g", "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
    frontier = []

    ####
    expanded = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
    while len (frontier)>0: #if we have elements in the frontier...
         # 3.1. get first element of frontier and delete it
        node = frontier[0]
        frontier = frontier[1:]

        #add to expanded--> we add the state, as the rest of fields will be different
        expanded.append(node["state"])

        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            # check if it is expanded before adding to frontier
            if not np.any ([np.all(n["state"]== e) for e in expanded]):
             ##### CHANGE THE CONDITION TO FIT ALL SIZES ARRAYS::
             # if the state of N is not any of the elements in expanded, add to the frontier
            #if n["state"] not in expanded:
                frontier.append(n)

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)

## IDS

In [ ]:
def IDS(problem, depth_limit, iteration_limit):
    # result dictionary
    result = {"method":"IDS"+str(depth_limit), "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
    frontier = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1

    # we add a control of the accepted max depth, starting with 1:
    current_max_depth=1

    #change the loop condition, now we have to control the end of the loop inside it
    while True:
        #if the len is 0, we have reached to the end of the depth. We start again with an increased max depth
        if len (frontier)==0:
            if current_max_depth<depth_limit:
                current_max_depth+=1
                node ={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
                frontier.append(node)
            else:
                result["status"]= "No nodes in the frontier. No solution possible."
                break

        #control if we have reached the iteration limit, stop
        if iterations>iteration_limit:
            result["status"] = "Maximum number of iterations reached"
            break

         # 3.1. get LAST element of frontier and delete it
        node = frontier[-1]
        frontier = frontier[:-1]

        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            ##### WE ONLY APPEND IF DEPTH <= CURRENT MAX DEPTH
            if n["depth"]<=current_max_depth:
                frontier.append(n)
            #in other case node is not appended, ending the loop

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)

## IDS_g

In [ ]:
def IDS_g(problem, depth_limit, iteration_limit):
    # result dictionary
    result = {"method":"IDS_g-"+str(depth_limit), "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
    frontier = []

    ####
    expanded = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
     # we add a control of the accepted max depth, starting with 1:
    current_max_depth=1

    #change the loop condition, now we have to control the end of the loop inside it
    while True:
        #if the len is 0, we have reached to the end of the depth. We start again with an increased max depth
        if len (frontier)==0:
            if current_max_depth<depth_limit:
                current_max_depth+=1
                node ={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
                frontier.append(node)
                expanded = []
            else:
                result["status"]= "No nodes in the frontier. No solution possible."
                break
       #before doing anything, if we have reached the iteration limit, stop
        if iterations>iteration_limit:
            result["status"] = "Maximum number of iterations reached"
            break
         # 3.1. get first element of frontier and delete it
        node = frontier[-1]
        frontier = frontier[:-1]

        #add to expanded--> we add the state, as the rest of fields will be different
        expanded.append(node["state"])

        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            # check if it is expanded before adding to frontier
            if not np.any ([np.all(n["state"]== e) for e in expanded]):
            # if n["state"] not in expanded:
                if n["depth"]<=current_max_depth:
                    frontier.append(n)

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)

# INFORMED SEARCH




## GBFS: GREEDY BEST FIRST SEARCH

The algorithm is equal to BFS, but in this case we add a new step, sorting the frontier in order to get the most similar node the first to be expanded.

Besides, we need to add a control of iterations, as with other methods, as GBFS is not complete and might enter an infinite loop
```
1. Make a node with the initial problem state
2. Insert node into the frontier data structure
3. WHILE final state not found AND frontier is not empty DO
  3.1 Remove first node from the frontier
  3.2 IF node contains final state THEN final state found
  3.3 IF node doesn’t contain final state THEN
     3.3.1 EXPAND node’s state
     3.3.2 Insert successor nodes into frontier
     3.3.3 Sort frontier in ascending order of f(n)
4. IF final state found THEN
  4.1  RETURN sequence of actions found
5. ELSE  “solution not found”
```


In [ ]:
def GBFS(problem, iteration_limit):
    # result dictionary
    result = {"method":"GBFS", "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    # problem = Problem()
#####---->>>>>>> initial evaluation is obtained from the function
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":problem.get_evaluation(problem.initial_state)}
    frontier = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
    while len (frontier)>0: #if we have elements in the frontier...

######---->>>>>>>> iteration control
        #control if we have reached the iteration limit, stop
        if iterations>iteration_limit:
            result["status"] = "Maximum number of iterations reached"
            break

         # 3.1. get first element of frontier and delete it
        node = frontier[0]


        frontier = frontier[1:]
        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            frontier.append(n)

################# SORTING THE FRONTIER
        # sorting by ascending order this will lead to
        frontier = sorted(frontier, key=lambda field: field['evaluation'])

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)

## GBFS-Graph

In [ ]:
def GBFS_g(problem, iteration_limit):
    # result dictionary
    result = {"method":"GBFS", "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    # problem = Problem()
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":problem.get_evaluation(problem.initial_state)}
    frontier = []

####---->>> EXPANDED NODES LIST
    expanded = []
     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
    while len (frontier)>0: #if we have elements in the frontier...
       #control if we have reached the iteration limit, stop
        if iterations>iteration_limit:
            result["status"] = "Maximum number of iterations reached"
            break

         # 3.1. get first element of frontier and delete it
        node = frontier[0]


        frontier = frontier[1:]
        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
######----->>>>>>> check if it is expanded before adding to frontier
            if not np.any ([np.all(n["state"]== e) for e in expanded]):
              frontier.append(n)


        # sorting by ascending order this will lead to
        frontier = sorted(frontier, key=lambda field: field['evaluation'])

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)



# COMPARISON


In [ ]:
problem = ProblemP8()
def print_results (res):
  print ("METHOD: {} \n Final_state: {} \n Final Status: {} \n Maximum frontier size: {}  \n Maximum Depth: {} \n Iterations reached: {}".format(res["method"], res["final_state"]["actions"], res["status"], res["max_frontier"], res["max_depth"], res["iterations"]))


In [ ]:
bfs_res = BFS(problem)
print_results (bfs_res)

METHOD: BFS 
 Final_state: ['Right', 'Right', 'Up', 'Up', 'Left', 'Left'] 
 Final Status: Solution Found. 
 Maximum frontier size: 952  
 Maximum Depth: 6 
 Iterations reached: 541


In [ ]:
bfsg_res = BFS_g(problem)
print_results (bfsg_res)

METHOD: BFS_g 
 Final_state: ['Right', 'Right', 'Up', 'Up', 'Left', 'Left'] 
 Final Status: Solution Found. 
 Maximum frontier size: 66  
 Maximum Depth: 6 
 Iterations reached: 84


In [ ]:
ids_res     = IDS(problem, depth_limit = 8, iteration_limit = 1000)
print_results (ids_res )

METHOD: IDS8 
 Final_state: ['Right', 'Right', 'Up', 'Up', 'Left', 'Left'] 
 Final Status: Solution Found. 
 Maximum frontier size: 11  
 Maximum Depth: 6 
 Iterations reached: 380


In [ ]:
idsg_res  = IDS_g(problem, depth_limit = 8, iteration_limit = 1000)
print_results (idsg_res )

METHOD: IDS_g-8 
 Final_state: [] 
 Final Status: No nodes in the frontier. No solution possible. 
 Maximum frontier size: 2  
 Maximum Depth: 1 
 Iterations reached: 10


In [ ]:
gbfs = GBFS(problem, iteration_limit=35000)
print_results (gbfs )

KeyboardInterrupt: ignored

In [ ]:
gbfs_g= GBFS_g(problem, iteration_limit=35000)
print_results (gbfs_g )

METHOD: GBFS 
 Final_state: ['Up', 'Left', 'Left', 'Down', 'Up', 'Up', 'Right', 'Left', 'Down', 'Up', 'Right', 'Left', 'Down', 'Up', 'Down', 'Up', 'Down', 'Down', 'Up', 'Up', 'Down', 'Up'] 
 Final Status: Maximum number of iterations reached 
 Maximum frontier size: 49958  
 Maximum Depth: 24 
 Iterations reached: 35000
